# 1) NER + NEL

In [1]:
%%capture
!pip install spacy
!pip install spacy-dbpedia-spotlight 
!pip install wikipedia

In [2]:
import spacy
from spacy.cli import download as spacy_download
import wikipedia
import pandas as pd

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [3]:
# Scarico modello
spacy_model = 'en_core_web_trf'
spacy_download(spacy_model)

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')


In [4]:
%%capture
!pip install spacy-transformers 
import spacy_transformers

# Carico modello 
ner = spacy.load(spacy_model)

In [5]:
try:
    ner.add_pipe('dbpedia_spotlight')
    ner.pipeline
except:
    print("pipeline already present")

In [6]:
def getLink(question):
  doc_spotlight = ner(question)
  doc_spotlight.ents

  df = pd.DataFrame(
      [(ent.text, ent.kb_id_, ent._.dbpedia_raw_result['@similarityScore']) for ent in doc_spotlight.ents],
      columns = ['mention', 'link', 'similarity']
  )

  entity = df["link"][0]
  print(entity)
  return entity

In [7]:
%%capture
!pip install SPARQLWrapper

from SPARQLWrapper import SPARQLWrapper, JSON
sparql = SPARQLWrapper("http://dbpedia.org/sparql")

In [8]:
# Prendo abstract da entità
def abstractFromEntity(entity):
	sparql.setQuery("""
prefix dbpedia: <http://dbpedia.org/resource/>
prefix dbpedia-owl: <http://dbpedia.org/ontology/>
select ?abstract where { 
  <"""+ entity +"""> dbpedia-owl:abstract ?abstract ;
                                         dbpedia-owl:thumbnail ?thumbnail .
filter(langMatches(lang(?abstract),"en"))
}
  """)

	sparql.setReturnFormat(JSON)
	return sparql.query().convert()
 


def query2Text(results = None):
	text = ""
	if results != None:
		for result in results["results"]["bindings"]:
			text = text + result["abstract"]["value"].replace('\n', ' ').replace('\r', '') 
	return text

In [27]:
# question
question1 = "Which team does Max Verstappen compete for?"
question2 = "who is max verstappen father?"
question3 = "who is max verstappen?"
question8 = "when max verstappen won his first race?"
question9 = "who is kimi raikkonnen?"
question = question8

In [10]:
entity = getLink(question)

http://dbpedia.org/resource/Max_Verstappen


In [11]:
abstract = abstractFromEntity(entity)
abstract = query2Text(abstract)
abstract

"Max Emilian Verstappen (born 30 September 1997) is a Belgian-Dutch racing driver and the 2021 Formula One World Champion. He competes under the Dutch flag in Formula One with Red Bull Racing. He is the son of former Formula One driver Jos Verstappen. At the 2015 Australian Grand Prix, when he was aged 17 years, 166 days, he became the youngest driver to compete in Formula One. After spending the 2015 season with Scuderia Toro Rosso, Verstappen started his 2016 campaign with the Italian team before being promoted to parent team Red Bull Racing after four races as a replacement for Daniil Kvyat. At the age of 18, he won the 2016 Spanish Grand Prix on his debut for Red Bull Racing, becoming the youngest-ever driver and the first Dutch driver to win a Formula One Grand Prix. After winning the 2021 Abu Dhabi Grand Prix, Verstappen became the first Dutch driver to win the Formula One World Championship, and the 34th Formula One World Drivers' Champion. As of the 2022 Italian Grand Prix, Ver

# 2) LM + Answer 

In [12]:
from transformers import AutoTokenizer, TFAutoModelForQuestionAnswering
import tensorflow as tf

In [13]:
tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
model = TFAutoModelForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

In [14]:
def answerQuestion(question, text):

  inputs = tokenizer(question, text, add_special_tokens=True, return_tensors="tf",truncation=True)

  input_ids = inputs["input_ids"].numpy()[0]
      
  outputs = model(inputs)
  answer_start_scores = outputs.start_logits
  answer_end_scores = outputs.end_logits
      
  answer_start = tf.argmax(
      answer_start_scores, axis=1
  ).numpy()[0]  # Get the most likely beginning of answer with the argmax of the score
  answer_end = (
      tf.argmax(answer_end_scores, axis=1) + 1
  ).numpy()[0]  # Get the most likely end of answer with the argmax of the score
  answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))
      
  print(f"Question: {question}")
  print(f"Answer: {answer}")
  

In [28]:
answerQuestion(question, abstract)

Question: when max verstappen won his first race?
Answer: 2016 spanish grand prix
